In [4]:
from keras.src.models import Sequential
from keras.src.layers import Dense, Flatten, Conv2D, Dropout, Input, AveragePooling2D, GlobalAveragePooling2D
from keras.src.optimizers import Adam
from keras.src.callbacks import EarlyStopping
import os
import keras
import matplotlib.pyplot as plt
import glob
from keras.src.applications.resnet import ResNet50

In [7]:
data_path = 'data'
dirs = os.listdir(data_path)

In [5]:
train, validation = keras.utils.image_dataset_from_directory(directory='data', 
                                                labels='inferred',
                                                label_mode='int',
                                                image_size=(256,256),
                                                seed=2009,
                                                validation_split=0.15,
                                                subset='both',
                                                verbose=True)

Found 30965 files belonging to 7 classes.
Using 26321 files for training.
Using 4644 files for validation.


In [6]:
train_set = train.take(25_005)
test_set = train.skip(25_005)

In [30]:
# my neural network
models = Sequential(
    layers = [
        Input(shape=(256, 256, 3), batch_size=32),
        Conv2D(16, (7, 7), activation='relu'),
        AveragePooling2D((2, 2)),
        Conv2D(32, (7, 7), activation='relu'),
        AveragePooling2D((2, 2)),
        Conv2D(32, (5, 5), activation='relu'),
        AveragePooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dense(len(dirs), activation='softmax')
    ],
    name='softmax_model'
)
print("network created")

network created


In [31]:
# Compiling the model
models.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("model compiled")

model compiled


In [32]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
n_epoche = 30

In [33]:
models.summary()

Model: "softmax_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (32, 250, 250, 16)     │         2,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_12            │ (32, 125, 125, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (32, 119, 119, 32)     │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_13            │ (32, 59, 59, 32)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (32, 55, 55, 32)       │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_14            │ (32, 27, 27, 32)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (32, 25, 25, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (32, 40000)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (32, 40000)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (32, 256)              │    10,240,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (32, 7)                │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,313,671 (39.34 MB)

 Trainable params: 10,313,671 (39.34 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# Fitting the model
history = models.fit(train_set, epochs=n_epoche, validation_data=(validation), batch_size=32, callbacks=[early_stopping])
print("model fitted")

Epoch 1/30
823/823 ━━━━━━━━━━━━━━━━━━━━ 1072s 1s/step - accuracy: 0.2403 - loss: 6.0353 - val_accuracy: 0.3594 - val_loss: 1.6327
Epoch 2/30
823/823 ━━━━━━━━━━━━━━━━━━━━ 1091s 1s/step - accuracy: 0.3896 - loss: 1.5691 - val_accuracy: 0.3805 - val_loss: 1.5689
Epoch 3/30
823/823 ━━━━━━━━━━━━━━━━━━━━ 1105s 1s/step - accuracy: 0.4283 - loss: 1.4884 - val_accuracy: 0.4132 - val_loss: 1.5227
Epoch 4/30
823/823 ━━━━━━━━━━━━━━━━━━━━ 1184s 1s/step - accuracy: 0.4683 - loss: 1.3888 - val_accuracy: 0.4352 - val_loss: 1.4990
Epoch 5/30
823/823 ━━━━━━━━━━━━━━━━━━━━ 1188s 1s/step - accuracy: 0.4986 - loss: 1.3116 - val_accuracy: 0.4645 - val_loss: 1.4642
Epoch 6/30
 96/823 ━━━━━━━━━━━━━━━━━━━━ 16:49 1s/step - accuracy: 0.5416 - loss: 1.2126

KeyboardInterrupt: 

In [ ]:
# pre trained neural network: ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
models = Sequential(
    layers = [
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(len(dirs), activation='softmax')
    ],
    name='resnet'
)